In [32]:
from IPython.display import display, Math
import numpy as np
import math
import itertools 

# TODO: make : if vars > len(standart_var_names) then vars = x_1, x_2 ... x_n
standart_var_names = 'xyzpqtuv'
file_name = 'task2_1.txt'

In [43]:
class Monomial:
    """Auxiliary class. Used for creating and representation polynomials in fancy way.
    Especially useful for input polynomials by hand"""

                           
    def __init__(self, coeff, exponents):
        self.coefficient = coeff
        self.exponents = list(exponents)

    def __str__(self):
        coef = ''
        if np.count_nonzero(np.array(self.exponents)) == 0:
            return str(self.coefficient)
        elif self.coefficient == -1:
            coef = '-'
        elif self.coefficient != 1:
            coef = str(self.coefficient)
        return coef + \
            "".join([f'{standart_var_names[i]}{"^{" + str(int(self.exponents[i])) + "}" if self.exponents[i]!= 1 else ""}'
                      if self.exponents[i]!=0 else ''
                     for i in range(len(self.exponents))])

    
    
    # DEPRECATED . Now all arifmetical operations apply only on Polynomials! 
    
#     def __mul__(self, other):
#         exponents = list(np.array(self.exponents) + np.array(other.exponents))
#         coefficient = self.coefficient * other.coefficient
#         return Monomial(coefficient, exponents)
        
#     def __truediv__(self, other):
#         exponents = list(np.array(self.exponents) - np.array(other.exponents))
#         coefficient = self.coefficient / other.coefficient
#         return Monomial(coefficient, exponents)

#     def __neg__(self):
#         return Monomial(-self.coefficient, self.exponents)
    
#     def __sub__(self, other):
#         return self + (-other)

#     def __add__(self, other):
#         if type(other) is int:
#             return Polynomial([self, Monomial(coeff=other, exponents=[0] * len(self))]) if other != 0 else self
#         elif self.is_zero():
#             return other
#         elif other.is_zero():
#             return self
#         elif self.exponents == other.exponents:
#             return Monomial(coeff = self.coefficient + other.coefficient, exponents=self.exponents)
#         else:
#             return Polynomial([self, other])
        
    
#     def __eq__(self, other):
#         return self.exponents == other.exponents and self.coefficient == other.coefficient
    
    def gcd(self, other):
        coefficient = math.gcd(self.coefficient, other.coefficient)
        greatest_common_divisor_powers = np.minimum(np.array(self.exponents), np.array(other.exponents))
        return Monomial(coefficient, list(greatest_common_divisor_powers))
        
    def is_const(self):
        return np.count_nonzero(np.array(self.exponents)) == 0
    
    def print(self):
         display(Math(str(self)))
        
    def is_zero(self):
        if self.coefficient == 0:
            return True
        


In [42]:
class Polynomial:
    monomials = []
    matrix = np.zeros([])
    
    def sort_monomials(self):
        """Sorting monomials in lexicographic order"""
        self.monomials.sort(key = lambda monome : monome.exponents, reverse=True)
        return self

    def leading_term(self):
        return self.monomials[0]
    
    def __init__(self, monomials = None, matrix = None):
        if matrix is None:
            if type(monomials) is not list:
                monomials = [monomials]
            self.monomials = monomials
            self.sort_monomials()
            matrix_form=[]
            for monome in self.monomials:
                matrix_form.append([monome.coefficient] + monome.exponents)
            self.matrix = np.array(matrix_form)
        else:
            self.monomials = []            
            for row in matrix:
                self.monomials.append(Monomial(coeff=row[0], exponents=list(row[1:])))
            self.sort_monomials()
            self.matrix = matrix
        
    def print(self):
        return display(Math(str(self)))

    def __str__(self):
        return "+".join([str(monomial) for monomial in self.monomials]).replace('+-', '-')
    
    def __mul__(self, other, verbose = False):
        m1 = self.matrix
        m2 = other.matrix
        
        num_monomes1 = m1.shape[0] 
        num_monomes2 = m2.shape[0]
        
        n_variables = m1.shape[1] - 1
        result = np.zeros(shape=(num_monomes1*num_monomes2, n_variables + 1))
        
        idx=0
        for row1 in m1:
            for row2 in m2:
                coef1=row1[0]
                coef2=row2[0]
                exp1=row1[1:]
                exp2=row2[1:]
                result[idx, 0] = coef1 * coef2
                result[idx, 1:] = exp1 + exp2
                idx +=1
                
        if verbose:
            print('Multiplying polynomials...')
            self.print()
            print('and')
            other.print()
            print(f'First polynome matrix = {m1},\nSecond polynome matrix = {m2}\n')
            print(f'After multiplication:\n{result}')
        result=self.simplify(result)
        if verbose:
            print(f'After simplification:\n{result}')
            
        return Polynomial(monomials=None, matrix=result)
    
    def simplify(self, matrix, verbose = True):
        coeff=matrix[:,0]
        exp = matrix[:,1:]
        unique_exp, count = np.unique(exp, axis=0, return_counts=True)
        
#         if unique_exp.shape[0] == matrix.shape[0]:
#             return matrix
        
        new_matrix = np.vstack((np.zeros(unique_exp.shape[0]), unique_exp.T)).T
                
        for idx, r_row in enumerate(unique_exp):
            mask = np.flatnonzero((exp == r_row).all(axis=1))
            new_coef = sum(coeff[mask])
            new_matrix[idx, 0] = new_coef
            
        new_matrix=new_matrix[new_matrix[:,0]!=0]
        
        if verbose:
            print(f'Matrix:\n{matrix}\nUnique exponents:\n{unique_exp},\nNew matrix:\n{new_matrix}\n')

            
        return new_matrix

    
    def __truediv__(self, other, verbose=False):
        m1 = self.matrix
        m2 = other.matrix
        
        result = np.zeros(shape=(m1.shape[0] * m2.shape[0], m1.shape[1]))
        
        idx=0
        for row1 in m1:
            for row2 in m2:
                result[idx, 0] = row1[0] / row2[0]
                result[idx, 1:] = row1[1:] - row2[1:]
                idx +=1
                
        if verbose:
            print('Division of polynomials...')
            self.print()
            print('and')
            other.print()
            print(f'First polynome matrix = {m1},\nSecond polynome matrix = {m2}\n')
            print(f'After multiplication:\n{result}')
        result=self.simplify(result)
        if verbose:
            print(f'After simplification:\n{result}')
            
        return Polynomial(monomials=None, matrix=result)

    def __sub__(self, other):
        return self + (-other)

    def __add__(self, other):
        m1 = self.matrix
        m2 = other.matrix
        result = np.vstack((m1, m2))
        return Polynomial(matrix=self.simplify(result))
        
    def __neg__(self):
        matrix = np.array(self.matrix, copy=True)
        matrix[:,0] = -matrix[:,0]
        return Polynomial(matrix=matrix)

    def is_zero(self):
        for monom in self.monomials:
            if not monom.is_zero():
                return False
        return True

    def is_const(self):
        for monom in self.monomials:
            if not monom.is_const():
                return False
        return True
    
    def reduction(self, basis, verbose=True):
        reducible_polynomial = self
        sequence_of_reduction = [reducible_polynomial]
        
#         if type(reducing_poly) is not Polynomial:
#             reducing_poly=Polynomial(reducing_poly)
        
        if reducible_polynomial.is_const():
            return reducing_poly
        
        if verbose:
            print(f'Reducing polynomial : {str(reducible_polynomial)} with basis ({", ".join([str(poly) for poly in basis])})')
        
#         for poly_from_basis in basis:
#             Q = Polynomial(reducible_polynomial.leading_term()) / Polynomial(poly_from_basis.leading_term())
            
#             if verbose:
#                 print(f'Reducing polynomial : {str(reducible_polynomial)} with polynomial from basis {str(poly_from_basis)}')
#                 print(f'f_iC : {str(reducible_polynomial.leading_term())}, f_jC : {str(poly_from_basis.leading_term())}, Q = {str(Q)}')
            
#             if (Q.matrix[:, 1:] >= 0).all(): # If leading term of poly which is been reducing is divided by leading term of poly from basis
#                 reducible_polynomial = reducible_polynomial - Q * poly_from_basis
#                 # Если поделился - сделать редукцию, а потом сначала поделить на ВСЕ многочлены базиса
#                 # проверить чтобы базис был базисом, а не self.poly
#                 sequence_of_reduction.append(reducing_poly)
#             else:
#                 continue
                
        idx = 0
        print(len(basis))
        while idx < len(basis):
            print(f'idx = {idx}')
            reducible_polynomial.print()
            basis[idx].print()
            Q = Polynomial(reducible_polynomial.leading_term()) / Polynomial(basis[idx].leading_term())
            if (Q.matrix[:, 1:] >= 0).all(): # If leading term of poly which is been reducing is divided by leading term of poly from basis
                reducible_polynomial = reducible_polynomial - Q * basis[idx]
                print('reduced poly : ')
                reducible_polynomial.print()
                sequence_of_reduction.append(reducible_polynomial)
                idx = 0
            idx += 1
        
        if verbose:
            print('Sequence of reductions of polynomial:')
            display(Math(" \Rightarrow ".join([str(poly) for poly in sequence_of_reduction])))
                
        return sequence_of_reduction[-1]

In [44]:
poly = Polynomial([Monomial(1, [0, 0, 0]), Monomial(1, [2, 4, 1])])
poly.print()
simplified = Polynomial(matrix=poly.simplify(poly.matrix))
simplified.print()
str(simplified)

<IPython.core.display.Math object>

Matrix:
[[1 2 4 1]
 [1 0 0 0]]
Unique exponents:
[[0 0 0]
 [2 4 1]],
New matrix:
[[1. 0. 0. 0.]
 [1. 2. 4. 1.]]



<IPython.core.display.Math object>

'x^{2}y^{4}z+1.0'

In [45]:
poly1 = Polynomial([Monomial(1, [2, 1, 1]), Monomial(2, [0, 1, 0])])
poly2 = Polynomial([Monomial(1, [2, 1, 1]), Monomial(2, [0, 1, 0])])
poly1.print()
poly2.print()
(poly1/poly2).print()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Matrix:
[[ 1.   0.   0.   0. ]
 [ 0.5  2.   0.   1. ]
 [ 2.  -2.   0.  -1. ]
 [ 1.   0.   0.   0. ]]
Unique exponents:
[[-2.  0. -1.]
 [ 0.  0.  0.]
 [ 2.  0.  1.]],
New matrix:
[[ 2.  -2.   0.  -1. ]
 [ 2.   0.   0.   0. ]
 [ 0.5  2.   0.   1. ]]



<IPython.core.display.Math object>

In [ ]:
poly1 = Polynomial([Monomial(3, [2, 1, 1]), Monomial(2, [0, 1, 0])])
poly2 = Polynomial([Monomial(2, [0, 1, 1]), Monomial(1, [1, 0, 0])])
poly3 = Polynomial([Monomial(3, [0, 1, 1])])
poly1.print()
poly2.print()
poly3.print()

poly3.reduction([poly1, poly2])



In [ ]:
def parse_file(file_path):
    f = open(file_path)
    m_equations, n_variables = [int(x) for x in f.readline().split()]
    polynomes = []
    for i in range(m_equations):
        cur_line = f.readline().split()
        num_of_monomes = int(cur_line[0])
        monomes = []
        for j in range(num_of_monomes):
            shift = n_variables + 1
            cur_substr = cur_line[1 + j*shift:1 + j*shift + shift]
            monomes.append(Monomial(coeff=int(cur_substr[0]), exponents=[int(x) for x in cur_substr[1:]]))
        polynomes.append(Polynomial(monomials=monomes))
    SAU = SystemOfEquations(polynomes)
    display(Math(str(SAU)))
    return SAU
    


In [36]:
# Demonstration of simplification

bad_poly = Polynomial([Monomial(0, [1, 1, 1]), Monomial(1, [23, 4, 0])])
bad_poly.print()
Polynomial(matrix=bad_poly.simplify(bad_poly.matrix)).print()

bad_poly2 = Polynomial([Monomial(2, [1, 1, 1]), Monomial(3, [1, 1, 1])])
bad_poly2.print()
Polynomial(matrix=bad_poly.simplify(bad_poly2.matrix)).print()

<IPython.core.display.Math object>

Matrix:
[[ 1 23  4  0]
 [ 0  1  1  1]]
Unique exponents:
[[ 1  1  1]
 [23  4  0]],
New matrix:
[[ 1. 23.  4.  0.]]



<IPython.core.display.Math object>

<IPython.core.display.Math object>

Matrix:
[[2 1 1 1]
 [3 1 1 1]]
Unique exponents:
[[1 1 1]],
New matrix:
[[5. 1. 1. 1.]]



<IPython.core.display.Math object>

In [ ]:
# Demonstation of arithmetical operations on polynomials

poly1 = Polynomial([Monomial(3, [2, 1, 1]), Monomial(2, [0, 1, 0])])
poly2 = Polynomial([Monomial(2, [2, 1, 1]), Monomial(1, [1, 0, 0])])
poly1.print()
poly2.print()
(poly1+poly2).print()
(poly1-poly2).print()
(poly1*poly2).print()
(poly1/poly2).print()
gcd = Polynomial([poly1.monomials[0].gcd(poly2.monomials[0])])
print('GCD of leading terms of poly1 and poly2 :')
gcd.print()
(poly1/gcd).print()
(poly2/gcd).print()

In [ ]:
# Demonstation of arithmetical operations on polynomials

poly1 = Polynomial([Monomial(3, [2, 1, 1]), Monomial(2, [0, 1, 0])])
poly2 = Polynomial([Monomial(2, [1, 1, 1]), Monomial(1, [1, 0, 0])])
poly1.print()
poly2.print()
(poly1*poly2).print()
gcd = poly1.monomials[0].gcd(poly2.monomials[0])
gcd.print()
(poly1/Polynomial([gcd])).print()

In [ ]:
# Demonstation of arithmetical operations on polynomials

poly1 = Polynomial([Monomial(3, [2, 1, 1]), Monomial(2, [0, 1, 0])])
poly2 = Polynomial([Monomial(-1, [1, 0, 0]), Monomial(1, [1, 4, 0])])
poly1.print()
poly2.print()
(poly1*poly2).print()
(-poly1).print()
(poly1/poly2).print()

In [ ]:
# Demonstation of sorting monomials in lexicographic order (Note: by default sorting is applied automatically when you
# create new instance of Polynomial class. If you want to see difference, you are supposed to comment corresponding line 
# in constructor of Polynomial class).

examplePolynome2 = Polynomial([Monomial(1, [0, 2, 3]), Monomial(1, [1, 0, 0]), Monomial(1, [0, 1, 3]), Monomial(1, [1, 1, 1]),
                              Monomial(1, [1, 2, 4]), Monomial(1, [3, 0, 0])])

display(Math(str(examplePolynome)))
display(Math(str(examplePolynome.sort_monomials())))

display(Math(str(examplePolynome2)))
display(Math(str(examplePolynome2.sort_monomials())))



In [ ]:
class SystemOfEquations:
    poly = []

    def __init__(self, polynomials):
        self.poly = polynomials
        self.sort_polynomes()
        
    def __str__(self):
        return '(' + ", ".join([str(polynome) for polynome in self.poly]) + ')'
        
    def print(self):
        display(Math(str(self)))
    
    def sort_polynomes(self):
        self.poly.sort(key = lambda polynome : polynome.monomials[0].exponents, reverse=True)
        return self
    
    def monomial_reduction(self, reduced_monomial, reducing_polynomial, verbose=True):
        Q = Polynomial(reduced_monomial) / Polynomial(reducing_polynomial.leading_term())
        
        if (Q.matrix[:, 1:] >= 0).all(): # If reducing poly divided by 
                reducing_poly = Polynomial(reduced_monomial) - Q * reducing_polynomial    
                
        if len(reducing_poly.monomials) > 1:
            raise ValueError("Sorry, result is not monomial")
        else:
            return Monomial(coeff=reducing_poly.monomials[0].coefficient, exponents=reducing_poly.monomials[0].exponents)
    
    # TODO: Перенести редукцию полиномов и мономов в класс полиномов 
    def polynomial_reduction(self, reducible_polynomial, basis, verbose=True):
        sequence_of_reduction = [reducible_polynomial]
        
#         if type(reducing_poly) is not Polynomial:
#             reducing_poly=Polynomial(reducing_poly)
        
        if reducible_polynomial.is_const():
            return reducing_poly
        
        if verbose:
            print(f'Reducing polynomial : {str(reducible_polynomial)} with basis {str(basis)}')
        
#         for poly_from_basis in basis:
#             Q = Polynomial(reducible_polynomial.leading_term()) / Polynomial(poly_from_basis.leading_term())
            
#             if verbose:
#                 print(f'Reducing polynomial : {str(reducible_polynomial)} with polynomial from basis {str(poly_from_basis)}')
#                 print(f'f_iC : {str(reducible_polynomial.leading_term())}, f_jC : {str(poly_from_basis.leading_term())}, Q = {str(Q)}')
            
#             if (Q.matrix[:, 1:] >= 0).all(): # If leading term of poly which is been reducing is divided by leading term of poly from basis
#                 reducible_polynomial = reducible_polynomial - Q * poly_from_basis
#                 # Если поделился - сделать редукцию, а потом сначала поделить на ВСЕ многочлены базиса
#                 # проверить чтобы базис был базисом, а не self.poly
#                 sequence_of_reduction.append(reducing_poly)
#             else:
#                 continue
                
            idx = 0
            while idx < len(basis):
                Q = Polynomial(reducible_polynomial.leading_term()) / Polynomial(basis[idx].leading_term())
                if (Q.matrix[:, 1:] >= 0).all(): # If leading term of poly which is been reducing is divided by leading term of poly from basis
                    reducible_polynomial = reducible_polynomial - Q * basis[idx]
                    sequence_of_reduction.append(reducible_polynomial)
                    idx = 0
                idx += 1
        
        if verbose:
            print('Sequence of reductions of polynomial:')
            display(Math(" \Rightarrow ".join([str(poly) for poly in sequence_of_reduction])))
                
        return sequence_of_reduction[-1]
                
    
    def Gröbner_basis(self, verbose = True):
        """Using Buchberger's algorithm"""
                
        if len(self.poly) == 1:
            return self.poly
        elif len(self.poly) == 0:
            raise ValueError("Current system of equations is empty")
            
        gröbner = self.poly.copy()

        possible_pairs_of_polynomes = list(itertools.combinations(gröbner, 2))
        message = rf'\text{{Find Gröbner basis of {str(self)} using Buchberger algorithm}} \\ '
        for poly1, poly2 in possible_pairs_of_polynomes:
            message += rf' \text{{Checking polynomes }} {str(poly1)} \text{{ and }} {str(poly2)}'
            ld_monom1 = poly1.leading_term()
            ld_monom2 = poly2.leading_term()
            gcd = ld_monom1.gcd(ld_monom2)
            if gcd.is_const():
                message += rf'\text{{, no link found}}'
            else :
                w = Polynomial([gcd])
                q1 = Polynomial([ld_monom1]) / w
                q2 = Polynomial([ld_monom2]) / w
                F_ij = poly1 * q2 - poly2 * q1
                result = self.polynomial_reduction(F_ij, gröbner)
                message += rf'\text{{, link found. Their leading terms is divided by }} {w}. \\ \text{{Therefore, }} f_{{iC}}= '
                message += rf'wq_1 \text{{ in our case is }} {str(ld_monom1)} = {str(w)}\cdot {str(q1)} '
                message += rf'\text{{ and }} f_{{jC}} = wq_2 \text{{ in our case is }} {str(ld_monom2)} = {str(w)}\cdot {str(q2)} \\ '
                message += rf'F_ij = f_i \cdot q_2 - f_j \cdot q_1 = ({str(poly1)}) \cdot ({str(q2)}) - ({str(poly2)}) \cdot ({str(q1)}) = {str(F_ij)} \\ '
                if result.is_zero() or result == 0:
                    message += rf'F_ij \text{{ reduced to }} {str(result)} \text{{. Nothing is added to basis}}'
                else:
                    possible_pairs_of_polynomes += [(result, poly_from_basis) for poly_from_basis in gröbner if poly_from_basis!=poly1 and poly_from_basis!=poly2]                    
                    gröbner.append(result)
                    message += rf'F_ij \text{{ reduced to }} {str(result)} \text{{ which is non-zero polynomial. It is added to basis. }}'                    
                    message += rf'\text{{ New basis: }} {str(SystemOfEquations(gröbner))} '
            message += rf'\\'
        message += rf'\text{{Result : Gröbner basis : }} {str(SystemOfEquations(gröbner))}'
        if verbose:
            display(Math(message))
        return SystemOfEquations(gröbner)
                       
    def reduced_Gröbner_basis(self, verbose=True):
        """Creating reduced Gröbner basis of initial basis using reduction algorithm"""
                
        def delete_multiple_element(list_object, indices):
            """Auxiliary function for deleting elements from list using list of indices (changes original list)
            :param list list_object : 
            :param list indices : 
            :return None
            """
            indices = sorted(indices, reverse=True)
            for idx in indices:
                if idx < len(list_object):
                    list_object.pop(idx)
        
        new_basis = self.Gröbner_basis(verbose=False).poly
        
        message = rf'\text{{Finding reduced Gröbner basis of }} {self}'

        
        # Minimization
        minimizing_indexes =[]
        for idx1 in range(len(new_basis)):
            flag_del_idx = False
            for idx2 in range(idx1 + 1, len(new_basis)):
                poly1 = new_basis[idx1]
                poly2 = new_basis[idx2]
                Q = Polynomial(poly1.leading_term()) / Polynomial(poly2.leading_term())
                if (Q.matrix[:, 1:] >= 0).all(): # If poly1 divided by poly2
                    flag_del_idx = True
            if flag_del_idx:
                minimizing_indexes.append(idx1)
                
        print(f'min_ind { minimizing_indexes}')
        
        delete_multiple_element(new_basis, minimizing_indexes)
        
        #Reduction of non-leading terms
        
        possible_pairs_of_polynomes = list(itertools.combinations(new_basis, 2))
        for poly1, poly2 in possible_pairs_of_polynomes:
            f_jC = Polynomial(poly2.leading_term())
            for monomial in poly1.monomials:
                Q = Polynomial(monomial) / f_jC
                if (Q.matrix[:, 1:] >= 0).all(): # If monomial divided by f_jC
                    # Found term of poly1 which is divided by leading term of poly2 
                    reduced_poly = self.monomial_reduction(monomial, poly2)
                    new_monomials = poly1.monomials.copy()
                    new_monomials[new_monomials.index(monomial)] = reduced_poly
                    new_poly1 = Polynomial(new_monomials)
                    new_basis[new_basis.index(poly1)] = new_poly1
        
        display(Math(rf'\text{{Reduced Gröbner basis : }} {str(SystemOfEquations(new_basis))}'))
        
        return SystemOfEquations(new_basis)
    
    def reduced_Gröbner_basis_built_in_func(self, auto_check = False):
        import wolframalpha
        query = (f'grobner basis {{{str(self)[1:-1]}}}')
        app_id = '3A76Y9-6PX7PH59P2'
        client = wolframalpha.Client(app_id)
        res = client.query(query)
        answer = next(res.results).text
        print('Reduced Gröbner basis computed by Wolfram Alpha:')
        display(Math(answer))
        if auto_check:
            print('Reduced Gröbner basis computed by my algorithm')
            display(Math(reduced_Gröbner_basis(self)))

In [ ]:
SAU_from_KR = SystemOfEquations([Polynomial([Monomial(1, [1, 1]), Monomial(1, [1, 0]), Monomial(-1, [0, 1])]), 
                                Polynomial([Monomial(1, [0, 1]), Monomial(1, [0, 0])])])
SAU_from_KR.print()

poly1 = SAU_from_KR.poly[0]

poly1.print()

SAU_from_KR.redu

In [ ]:
# Demonstation diffent operations on SAU and polynomials

SAU_from_KR = SystemOfEquations([Polynomial([Monomial(1, [1, 1]), Monomial(1, [1, 0]), Monomial(-1, [0, 1])]), 
                                Polynomial([Monomial(1, [0, 1]), Monomial(1, [0, 0])])])

SAU_from_KR.print()

poly1=SAU_from_KR.poly[0]
poly2=SAU_from_KR.poly[1]

poly1.print()
poly2.print()
(poly1+poly2).print()
(poly1-poly2).print()
(poly1*poly2).print()
(poly1/poly2).print()

ld_monom1 = poly1.leading_term()
ld_monom2 = poly2.leading_term()
gcd = ld_monom1.gcd(ld_monom2)

gcd.print()
w = Polynomial(gcd)
q1 = Polynomial(ld_monom1) / w
q2 = Polynomial(ld_monom2) / w
(poly1 * q2).print()
(poly2 * q1).print()
F_ij = poly1 * q2 - poly2 * q1
F_ij.print()

In [ ]:
# = SAU_from_KR.Gröbner_basis()
#.print()
SAU_from_KR.print()
SAU_from_KR.reduced_Gröbner_basis()
#.print()
#.poly[0].print()
SAU_from_KR.reduced_Gröbner_basis_built_in_func()


In [ ]:
SAU_from_file = parse_file(file_name)

SAU_from_file.Gröbner_basis()

SAU_from_file.reduced_Gröbner_basis()

SAU_from_file.reduced_Gröbner_basis_built_in_func()


Доп.тест: 

Система многочлены cyclic

f1 = a + b + c + d
f2 = ab + ad + bc + cd
f3 = abc + abd + acd + bcd
f4 = abcd - 1

In [ ]:
SAU_2 = SystemOfEquations([Polynomial([Monomial(14, [0, 2, 3]), Monomial(1, [1, 0, 0])]), 
                            Polynomial([Monomial(1, [30, 3, 3]), Monomial(1, [0, 1, 1])]),
                              Polynomial([Monomial(3, [1, 4, 2]), Monomial(1, [3, 0, 0])])])
display(Math(str(SAU_2)))
display(Math(str(SAU_2.sort_polynomes())))


In [ ]:
# DEPRECATED . Now all arifmetical operations apply only on Polynomials! 

m1 = Monomial(1, [1, 2, 3])
m2 = Monomial(1, [1, 0, 1])
display(Math(str(m1)))
display(Math(str(m2)))
display(Math(str(m1 / m2)))
display(Math(str(m2 / m1)))
display(Math(str(m1 * m2)))
display(Math(str(m1 + m2)))
display(Math(str(m1 - m2)))

(m1+m2-m1).print()

display(Math(str((m1+m2-m1).leading_term())))
p = Polynomial([m1, m2])

display(Math(str(p.leading_term())))

In [ ]:
exampleMonome1 = Monomial(coeff=5, exponents=[1, 2, 0, 4])
exampleMonome2 = Monomial(coeff=1, exponents=[0, 5, 2, 0])
print(exampleMonome1)
examplePolynome = Polynomial([exampleMonome1, exampleMonome2])
examplePolynome2 = Polynomial([Monomial(-7, [1, 0, 2, 9]), Monomial(-5, [1, 2, 3])])
example_SAU = SystemOfEquations([examplePolynome, examplePolynome2])

In [ ]:
display(Math(str(examplePolynome.sort_monomes())))

In [ ]:
display(Math(str(exampleMonome1)))
print(str(examplePolynome))
display(Math(str(examplePolynome)))
display(Math(str(example_SAU)))
